In [ ]:
%env DATA_DIR=$HOME/datasets
%env LOG_LEVEL=INFO
%env BATCH_SIZE=64
%env OPTIM_LR=0.001
%env OPTIM_M=0.8
%env TRAIN_EPOCHS=100
%env EXC_RESUME=1
%env DATASET_NAME=AMRB2_species
%env EMB_DIMS=512
%env MODEL_NAME=resnet

In [ ]:
import lightning.pytorch as pl
import numpy as np
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers.wandb import WandbLogger

from config import Config, load_config
from datasets import get_data
from models import get_model

In [ ]:
# initialize the RNG deterministically
np.random.seed(42)
torch.manual_seed(42)
torch.set_float32_matmul_precision('medium')

config = load_config()

# initialize data attributes and loaders
get_data(config)
config.print_labels()

assert config.datamodule

In [ ]:
model_randinit = get_model(config)

In [ ]:
from pathlib import Path
artifact_dir = WandbLogger.download_artifact(artifact="yasith/uq_project/model-33t7jols:best")
model_pretrain = model_randinit.load_from_checkpoint(Path(artifact_dir) / "model.ckpt", config=config)
model_pretrain.eval()
del model_randinit

In [ ]:
assert config.datamodule
config.datamodule.setup("test")

In [ ]:
test_loader = config.datamodule.test_dataloader()

In [ ]:
from torchmetrics import Accuracy
accuracy = Accuracy(task="multiclass", num_classes=len(config.get_ind_labels())).cuda()

In [ ]:
from models.common import edl_probs
from tqdm.auto import tqdm

# accuracy.reset()
classifier_loss = "edl"
for batch_idx, batch in enumerate(tqdm(test_loader)):
    x, y = batch
    print(x.size(), y.size())
#     x = x.cuda().float()
#     y = y.cuda().long()
    
#     z, logits, x_pred = model_pretrain(x)
    
#     # classifier loss
#     if classifier_loss == "edl":
#         pY, uY = edl_probs(logits)
#     elif classifier_loss == "crossent":
#         pY = logits.softmax(-1)
#         uY = 1.0 - pY.max(-1)
#     elif classifier_loss == "margin":
#         pY = logits.sigmoid()
#         uY = 1.0 - pY.max(-1)
#     else:
#         raise ValueError(classifier_loss)
#     accuracy.update(pY, y)
    
# print(accuracy.compute())